In [4]:
import numpy as np
import pandas as pd
import plotly.express as px
from tools import import_data

# Import the data from the csv files
df_calibration = import_data("calibration.csv")
df_sample = import_data("sample.csv")

# Calculate the known concentration of the calibration samples in mg/L
df_calibration["Concentration mg/L"] = 50 / df_calibration["Dilution"]

df_calibration.head(), df_calibration.tail()

(     Sample Well  Wavelength  Dilution  Absorbance  Concentration mg/L
 6984  Blank   E1       220.0       1.0       3.977                50.0
 7275  Blank   E3       220.0       1.0       3.393                50.0
 7566  Blank   E5       220.0       1.0       3.439                50.0
 6985  Blank   E1       222.0       1.0       3.618                50.0
 7276  Blank   E3       222.0       1.0       3.726                50.0,
      Sample Well  Wavelength  Dilution  Absorbance  Concentration mg/L
 4654     S1   C7       798.0     128.0       0.051            0.390625
 6982     S1  D11       798.0     128.0       0.051            0.390625
 2327     S1   B3       800.0     128.0       0.050            0.390625
 4655     S1   C7       800.0     128.0       0.049            0.390625
 6983     S1  D11       800.0     128.0       0.051            0.390625)

# Part 1

The following Beer-Lambert equation can be used to determine the relationship between absorbance ($A$) and concentration ($c$):

$$
A = \Epsilon\ c\ l
$$

where $\Epsilon$ is the absorptivity coefficient and $l$ is optical path length in cm. Both $A$ and $E$ are functions of wavelengths. For the experiments we conduct $l$ is equal to 1 cm. 

We assume that when measuring multiple chemicals their absorbances can be added such that for $N$ chemicals, $A = \sum_{i=1}^{N}A_i$. When measuring a sample in the spectrophotometer (chemical in a solvent), we also measure the solvent only and call this a Blank.

The measured absorbance (labelled $A_{obs}$) is the combined absorbence of all of the chemicals in the solvent (labelled $A_{Blank}$) and the the absorbence of the pigments of interest (labelled $A_{S1}$). Under the assumption above, that the absorbances simply add combination, $A_{obs} = A_{S1} + A_{Blank}$, the blank-corrected absorbance is

$$
A_{S1} = A_{Obs} - A_{Blank}
$$

To apply this correction function, I will average $A_{Blank}$ at each measured wavelength, and subtract it from $A_{Obs}$.



In [20]:
# Split the dataframes into signal and blank
df_S1 = df_calibration[df_calibration["Sample"] == "S1"]
df_blank = df_calibration[df_calibration["Sample"] == "Blank"]

# Calculate the mean absorbance for each wavelength in the Blank samples
mean_abs_blank = df_blank.groupby("Wavelength").mean("Absorbance")["Absorbance"]

df_S1["Absorbance Corrected"] = df_S1["Absorbance"] - mean_abs_blank[df_S1["Wavelength"]].values



C:\Users\jj5718\AppData\Local\Temp\ipykernel_24728\2011941074.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:
mean_abs_blank

Wavelength
220.0    3.603000
222.0    3.555333
224.0    4.068667
226.0    4.390333
228.0    4.937333
           ...   
792.0    0.049333
794.0    0.049333
796.0    0.049667
798.0    0.049667
800.0    0.048667
Name: Absorbance, Length: 291, dtype: float64

In [41]:
fig = px.scatter( df_S1, 
                 x = "Wavelength", 
                 y = "Absorbance", 
                 color = "Concentration mg/L", 
                 labels={
                     "Wavelength": "Wavelength (nm)",
                     "Absorbance": "Absorbance",
                     "Concentration mg/L": "Concentration (mg/L)"
                 },
                 title = "Raw absorbance curve for S1")
fig.show()

fig = px.line( mean_abs_blank, 
                 title = "Blank absorbance curve")
fig.show()

fig = px.scatter( df_S1, 
                 x = "Wavelength", 
                 y = "Absorbance Corrected", 
                 color = "Concentration mg/L", 
                 labels={
                     "Wavelength": "Wavelength (nm)",
                     "Absorbance Corrected": "Corrected Absorbance",
                     "Concentration mg/L": "Concentration (mg/L)"
                 },
                 title = "Corrected absorbance curve for S1")
fig.show()




In [47]:
# From the plots above we can see
# 1) There are anomalous readings at the start of the wavelength range < 340 nm, likely due to the limit of detection of the machine.
# 2) The absorbance generally increases with concentration as expected. 

# First I will limit the dataset to around the peak

df_S1 = df_S1[(df_S1["Wavelength"] > 400 ) & (df_S1["Wavelength"] < 650)] 

According to the Beer-Lambert equation, the absorbances can be used to determine the relationship between absorbance ($A$) and concentration ($c$):

$$
A = \Epsilon\ c\ l
$$
